<a href="https://colab.research.google.com/github/ZabalaGaston/HPC/blob/main/Zabala_Gaston_ejercicio_2_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 Introducción

Una matriz traspuesta es el resultado de reordenar la matriz original mediantel cambio de filas por columnas y las columnas por filas en una nueva matriz. [3][2]

<center>

>$A_{m,n} =
 \begin{pmatrix}
  a_{1,1} & a_{1,2} \\
  a_{2,1} & a_{2,2} 
 \end{pmatrix}$  $ =
 \begin{pmatrix}
  a_{1,1} & a_{2,1} \\
  a_{1,2} & a_{2,2} 
 \end{pmatrix}$ , 1 **<=** m, 1 **<=** j **<=** n
 </center> 

En este cuaderno, dejaré plasmado el algoritmo para trasponer una matriz cuadrada, utilizando GPU.

#2 Armado del ambiente

---
## 2.1 Instala en el cuaderno el módulo CUDA de Python.

In [2]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 12.0MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621146 sha256=08e21ffdce41229950acb0d303cc14b606a687ab323b75eb32cb073f590789fb
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=a978a51446f179909fd407354ed554c995b904fc7d432b2eb7b24514dad43d3f
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


---
# 3 Desarrollo
Ejecución del algoritmo para trasponer una matriz utilizando GPU.

In [2]:
#@title 3.1 Parámetros de ejecución { vertical-output: true }
#@title 3.1.1 Ingrese el orden { vertical-output: true }

ordenMatriz =   7200#@param {type:"slider", min:0, max:25000, step:100}

# --------------------------------------------
%matplotlib inline
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

tiempo_total = datetime.now()

# --------------------------------------------
#Funciones
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# Definición de función para crear matriz

def crearMatriz(orden):
  m = np.random.randint(100, 1000, (ordenMatriz, ordenMatriz))
  return m.astype( np.float32())

# CPU - Defino la función kernel que ejecutará en GPU.

module = SourceModule("""
__global__ void trasponerMatriz( int orden, float *mo, float *mt )
{
  // Calculo las coordenadas del Thread en dos dimensiones.
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  int idy = threadIdx.y + blockIdx.y*blockDim.y;

  // Verifico que los Thread estén dentro del orden de la matriz.
  if( idx < orden && idy < orden )
  {
    int indexMt =  idy * orden + idx;
    int indexMo =  idx * orden + idy;

    mt[indexMt] = mo[indexMo];
  }
}
""") 

# --------------------------------------------

# Creo las matrices
try:

  if ordenMatriz <= 0:
    raise ValueError("Error: el orden de la matriz debe ser mayor a 0");
  
  matriz_cpu = crearMatriz(ordenMatriz)
  matrizTraspuesta_cpu = crearMatriz(ordenMatriz)

  # Reservo los 2 vectores en GPU
  matriz_gpu = cuda.mem_alloc( matriz_cpu.nbytes )
  matrizTraspuesta_gpu = cuda.mem_alloc( matrizTraspuesta_cpu.nbytes )

  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod( matriz_gpu, matriz_cpu )
  cuda.memcpy_htod( matrizTraspuesta_gpu, matrizTraspuesta_cpu )

  # CPU - Genero la función kernel.
  kernel = module.get_function("trasponerMatriz")

  dim_hilo_x = 16
  dim_bloque_x = np.int( (ordenMatriz+dim_hilo_x-1) / dim_hilo_x )

  dim_hilo_y = 19
  dim_bloque_y = np.int( (ordenMatriz+dim_hilo_y-1) / dim_hilo_y )

  tiempo_traspuesta = datetime.now()

  kernel( np.int32(ordenMatriz), matriz_gpu, matrizTraspuesta_gpu, block=( dim_hilo_x, dim_hilo_y, 1 ), grid=(dim_bloque_x, dim_bloque_y,1) )

  tiempo_traspuesta = datetime.now() - tiempo_traspuesta

  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh( matrizTraspuesta_cpu, matrizTraspuesta_gpu )

  tiempo_total = datetime.now() - tiempo_total

  print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Tiempo Trasponer: ", tiempo_en_ms( tiempo_traspuesta ), "[ms]" )

  print("\n\nMatriz original\n", matriz_cpu)
  print("\nMatriz traspuesta\n",matrizTraspuesta_cpu)

except ValueError as vr:
  print(vr)
except Exception as e: 
  print(e.args)

Tiempo Total:  1033.062 [ms]
Tiempo Trasponer:  0.167 [ms]


Matriz original
 [[543. 306. 617. ... 599. 946. 746.]
 [377. 868. 304. ... 776. 493. 982.]
 [526. 308. 946. ... 787. 280. 770.]
 ...
 [718. 543. 879. ... 580. 829. 383.]
 [176. 602. 712. ... 326. 991. 250.]
 [412. 784. 623. ... 758. 866. 751.]]

Matriz traspuesta
 [[543. 377. 526. ... 718. 176. 412.]
 [306. 868. 308. ... 543. 602. 784.]
 [617. 304. 946. ... 879. 712. 623.]
 ...
 [599. 776. 787. ... 580. 326. 758.]
 [946. 493. 280. ... 829. 991. 866.]
 [746. 982. 770. ... 383. 250. 751.]]


---
# 4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      | pip install pycuda    | Instalar en el cuaderno los driver de CUDA para Python.
CPU      |  import                | Importar los módulos para funcionar.
CPU      |  datetime.now()        | Tomar el tiempo actual.
CPU      |  crearMatriz()         | Crear y cargar valores a la matriz original.
CPU      |  crearMatriz()         | Crear matriz traspuesta (matrizTraspuesta).
**GPU**  |  cuda.mem_alloc()      | Reservar la memoria para las matrices en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copiar los valores en crudo de las matrices al GPU.
CPU      |  SourceModule()        | Posee el código del kernel.
CPU      |  module.get_function() | Conviertir el texto del kernel en funcion de Python.
CPU      |  dim_hilo_x, dim_hilo_y| Calcular las dimensiones para la ejecuciòn de 2D.
**GPU**  |  kernel()              | Ejecutar el kernel en GPU, enviando los parametros (Traspone la matriz).
CPU      | datetime.now() - tiempo_traspuesta | Calcular tiempo total en trasponerMatriz. (GPU(
CPU      | cuda.memcpy_dtoh()     | Copiar desde la memoria GPU al CPU.
CPU      |  datetime.now() - tiempo_total | Calcular el tiempo total.
CPU      |  tiempo_en_ms( time ) | Transformar tiempos en milisegundos.
CPU      |  print()               | Mostrar estadísticas.
CPU      |  print()            | Mostrar matriz original y matriz traspuesta.


---
# 5 Conclusiones

Realicé la ejecución 10 veces para una matriz cuadrada de 10000 x 10000, obteniendo los siguientes resultados:

Tiempo Total:  1998.161 [ms] | Tiempo Trasponer:  0.174 [ms]<br>
Tiempo Total:  1911.462 [ms] | Tiempo Trasponer:  0.120 [ms]<br>
Tiempo Total:  1867.214 [ms] | Tiempo Trasponer:  0.112 [ms]<br>
Tiempo Total:  1897.369 [ms] | Tiempo Trasponer:  0.124 [ms]<br>
Tiempo Total:  1925.017 [ms] | Tiempo Trasponer:  0.119 [ms]<br>
Tiempo Total:  1944.206 [ms] | Tiempo Trasponer:  0.128 [ms]<br>
Tiempo Total:  1899.049 [ms] | Tiempo Trasponer:  0.126 [ms]<br>
Tiempo Total:  1924.625 [ms] | Tiempo Trasponer:  0.123 [ms]<br>
Tiempo Total:  1939.246 [ms] | Tiempo Trasponer:  0.130 [ms]<br>
Tiempo Total:  1904.423 [ms] | Tiempo Trasponer:  0.110 [ms]<br>


Podemos observar, que los tiempos de ejecución promedio [1920.8 ms] del programa son muchos **menores** a los obtenidos sin utilizar GPU. [56187.5 ms] , esto se debe a que al usar GPU la matriz se va recorriendo simultáneamente en varios threads.

Se puede apreciar como la ejecución de 2 dimensiones en paralelo, para este algoritmo, es mucho más rápida y eficiente con respecto a la secuencial. A  medida que aumenta el orden de las matrices, se puede valorar la diferencia entre ambas ejecuciones.


El punto más relevante en este ejercicio es la utilización de la función Kernel, la cuál nos permite ejecutar el algoritmo en dos dimensiones utilizando hilos.  Además, vale la pena mencionar los pasos que realizamos para reservar memoria en GPU (cuda.mem_alloc) y para copiar datos en memoria GPU (cuda.memcpy_htod).

En cuanto a lecciones aprendidas, comprendí el funcionamiento de hilos y la utilización del módulo CUDA.
Como mejora, se podrían agregar gráficos ilustrativos de las métricas obtenidas.


----
# 6 Bibliografía

[1] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Matriz traspuesta: [Enlace](https://es.wikipedia.org/wiki/Matriz_transpuesta)

[3] Traspuesta: [Enlace](https://economipedia.com/definiciones/matriz-traspuesta.html#:~:text=Una%20matriz%20traspuesta%20es%20el,filas%20en%20una%20nueva%20matriz)